In [1]:
from data_manipulation import*
from data_preprocessing import*
import pandas as pd
from fm_analycts_parametrs import DEF_STATS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# _df = fm_create_dataframe(path='data/italia_novo_2024.html')
_df = fm_create_dataframe(path='data/SERIEC_2024.html')

In [2]:
df = _df.copy()

In [14]:
sorted(df.columns.to_list())

['Melhor Pos',
 'Posição Sec.',
 'adef_c_p90',
 'adef_t_p90',
 'alivios_p90',
 'altura',
 'aof_p90',
 'ass',
 'ass_p90',
 'ass_per_aof',
 'aval_cria',
 'aval_def',
 'aval_fin',
 'bloqueios_p90',
 'cab_dec_p90',
 'cab_g_p100',
 'cab_g_p90',
 'cab_p_p90',
 'chutes_gol_p90',
 'chutes_p90',
 'clube',
 'coef',
 'conv_p100',
 'conv_penal_p100',
 'cruz_c_p100',
 'cruz_c_p90',
 'cruz_t_p90',
 'des_c_p100',
 'des_dec_p90',
 'des_g_p90',
 'des_t_p90',
 'divisao',
 'duel_g_p90',
 'duel_t_p90',
 'erros_decisivos_p90',
 'faltas_sofridas_p90',
 'final_contrato',
 'fintas_p90',
 'gc_per_aof',
 'gk_def_desv_p90',
 'gk_def_dif_p90',
 'gk_def_p90',
 'gk_def_segu_p90',
 'gk_gsof_p90',
 'gk_pen_def_p100',
 'gk_sg',
 'gk_xG_def',
 'gk_xG_def_p90',
 'gols',
 'grandes_chances',
 'grandes_chances_p90',
 'id',
 'id_temporada',
 'idade',
 'int_p90',
 'jg_ar_t_p90',
 'melhor_pe',
 'minutos',
 'motm',
 'nac',
 'nome',
 'nota_med',
 'npG',
 'npG_ae',
 'npG_p90',
 'npG_per_aof',
 'np_chutes',
 'np_chutes_gol',
 'np

In [5]:
df['jg_ar_t_p90'].describe()
df['adef_t_p90'].describe()

count    222.000000
mean      16.819054
std        2.169904
min       11.160000
25%       15.530000
50%       16.600000
75%       17.645000
max       23.890000
Name: adef_t_p90, dtype: float64

In [11]:
df['rtg_adef2'] = (
    df['rtg_adef']*0.75 + (df['poss_g_p90']/df['adef_t_p90'])*0.15 + np.tanh(df['adef_t_p90']/1)*0.10
).round(2)

In [16]:
df[['nome', 'idade','clube', 'divisao','rtg_jg_ar','rtg_duel','rtg_des','rtg_rec_bola','rtg_adef','rtg_adef2','passe_c_p100','pass_prog_p90','aval_def']].sort_values(by=['rtg_adef2'],ascending=False, ignore_index=True)[:20]

,nome,idade,clube,divisao,rtg_jg_ar,rtg_duel,rtg_des,rtg_rec_bola,rtg_adef,rtg_adef2,passe_c_p100,pass_prog_p90,aval_def
0,Éric Botteghin,36,Ascoli,Serie C NOW Girone B,0.58,0.40,0.70,1.28,0.66,0.79,86.0,3.87,0.77
1,Christian Pastina,23,Padova,Serie C NOW Girone A,0.60,0.58,0.65,1.02,0.71,0.79,92.0,3.84,0.74
2,Ivo Molnar,29,Caldiero,Serie C NOW Girone A,0.58,0.45,0.65,1.20,0.66,0.78,87.0,3.69,0.75
3,Andrea Tiritiello,29,Lucchese Libertas,Serie C NOW Girone B,0.53,0.51,0.62,1.19,0.65,0.77,89.0,4.24,0.74
4,Luka Koblar,24,Novara,Serie C NOW Girone A,0.60,0.48,0.69,1.08,0.68,0.77,88.0,4.21,0.74
5,Cristian Dell'Orco,30,Perugia,Serie C NOW Girone B,0.53,0.43,0.59,1.29,0.63,0.77,88.0,4.20,0.74
6,Roberto Pirrello,28,Gubbio,Serie C NOW Girone B,0.61,0.43,0.60,0.96,0.69,0.76,86.0,4.01,0.69
7,Riccardo Brosco,33,Pescara,Serie C NOW Girone B,0.56,0.45,0.66,1.07,0.66,0.76,91.0,3.72,0.71
8,Maxime Leverbe,27,Vicenza,Serie C NOW Girone A,0.57,0.47,0.63,1.13,0.66,0.76,90.0,4.14,0.73
9,Mario Noce,24,Legnago,Serie C NOW Girone A,0.64,0.46,0.60,0.86,0.70,0.75,89.0,4.19,0.68


In [ ]:
# df['divisao'].unique().tolist()

sorted(df.columns.to_list())

In [ ]:
df['aval_def_adj'] = df['aval_def']*df['coef']
df['rtg_jg_ar_adj'] = df['rtg_jg_ar']*df['coef']
df['rtg_des_adj'] = df['rtg_des']*df['coef']
df['rtg_adef_adj'] = df['rtg_adef']*df['coef']
df['rtg_duel_adj'] = df['rtg_adef']*df['coef']

p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj1'] = (((     (df['rtg_jg_ar_adj']*df['jg_ar_t_p90']*p1) +
                                        df['rtg_duel_adj']*p2 + 
                                        (df['rtg_des_adj']*df['des_t_p90'])*p3 + 
                                        (df['poss_g_p90']/df['adef_t_p90'])*p4+
                                        df['rtg_adef_adj']*p5
                                        ))).round(2)

In [ ]:
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj2'] = ((((df['rtg_jg_ar_adj']*p1) + df['rtg_duel_adj']*p2 + (df['rtg_des_adj'])*p3 + (df['poss_g_p90']/df['adef_t_p90'])*p4 + df['rtg_adef_adj']*p5))).round(2)

In [ ]:
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj3'] = ((((df['rtg_jg_ar']*p1) + df['rtg_duel']*p2 + (df['rtg_des'])*p3 + (df['poss_g_p90']/df['adef_t_p90'])*p4 + df['rtg_adef']*p5))).round(2)
df['aval_def_adj4'] = (df['aval_def_adj3']*df['coef']).round(2)

In [ ]:
df_zagueiros = df[df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]
df_zagueiros = df_zagueiros[df_zagueiros['divisao'] == 'Serie C NOW Girone C']
df_zagueiros['aval_def_adj4_pct'] = (df_zagueiros['aval_def_adj4'].rank(pct=True)).round(2)

In [ ]:
df_zagueiros[['nome', 'idade','clube', 'divisao','preco_min', 'posicao_analise', 'aval_def', 'aval_def_adj','aval_def_adj1','aval_def_adj2','aval_def_adj3','aval_def_adj4','aval_def_adj4_pct']].sort_values(by=['aval_def_adj3'],ascending=False, ignore_index=True)

In [ ]:
df_zagueiros['aval_def_adj4'].describe()

In [ ]:
div_med = _df.groupby('divisao')['aval_def'].mean()
div_desv = _df.groupby('divisao')['aval_def'].std()

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

_df_zagueiros['aval_def_adj'] = _df_zagueiros.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_adef_adj'] = _df_zagueiros.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_des_adj'] = _df_zagueiros.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_duel_adj'] = _df_zagueiros.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_jg_ar_adj'] = _df_zagueiros.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

_df_zagueiros['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df_zagueiros['aval_def_adj1'] = (((     (_df_zagueiros['rtg_jg_ar_adj']*_df_zagueiros['jg_ar_t_p90']*p1) +
                                        _df_zagueiros['rtg_duel_adj']*p2 + 
                                        (_df_zagueiros['rtg_des_adj']*_df_zagueiros['des_t_p90'])*p3 + 
                                        (_df_zagueiros['poss_g_p90']/_df_zagueiros['adef_t_p90'])*p4+
                                        _df_zagueiros['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df_zagueiros['aval_def_adj1'] = _df_zagueiros['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df_zagueiros['aval_def_adj2'] = (_df_zagueiros['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

print(_df_zagueiros.groupby('divisao')['rtg_adef'].mean())
print(_df_zagueiros.groupby('divisao')['rtg_adef'].std())

_df_zagueiros[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df_zagueiros[(_df_zagueiros['divisao'].str.contains('Serie C NOW')) & (_df_zagueiros['minutos'] >= 900) & (_df_zagueiros['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'minutos',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def'],ascending=False)

In [ ]:
_df['aval_def_adj'] = _df.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_adef_adj'] = _df.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_des_adj'] = _df.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_duel_adj'] = _df.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_jg_ar_adj'] = _df.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
_df['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df['aval_def_adj1'] = (((     (_df['rtg_jg_ar_adj']*_df['jg_ar_t_p90']*p1) +
                                        _df['rtg_duel_adj']*p2 + 
                                        (_df['rtg_des_adj']*_df['des_t_p90'])*p3 + 
                                        (_df['poss_g_p90']/_df['adef_t_p90'])*p4+
                                        _df['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df['aval_def_adj1'] = _df['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df['aval_def_adj2'] = (_df['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

In [ ]:
_df[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df[(_df['divisao'] == 'Serie A TIM') & (_df['minutos'] >= 900) & (_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def_adj2'],ascending=False)